# Email to interactive map

When I came to Paris as scientist, I asked support to the wonderful agency [Science Accueil](http://www.science-accueil.org). 
Their services are truly great and I easily got set-up in France.

However, reading their list of available flats was a nightmare, as  the list is truly a `textual list`:

    
>    **** HOUSING n° L xyz/uvw
>
>   Place du Pantheon
>   75005 Paris
>
>   DESCRIPTION OF THE HOUSING
>
>   1-room apartment in building on the 1ère étage

>   Living space: 7000 sq.m

>   This housing is suitable for 79 persons maximum.
>
>   KITCHEN
>
>   Lorem ipsum dolor  ..
>
>   BATHROOM(S)
>
>   Mauris maximus turpis ..
>
>   ...
 

If you do not know Paris (and I didn't), it is clearly difficult to understand the position and features of each flat, 
unless you make yourself a `map`.

## Python's automagic
Making a map of the flats and descriptions is so *booooring*.. let's make it automatic and fast!

The following code makes the map for you (jump at the end to see an example) in 5 simple steps
1. Parse the informations (as received, just copy and paste from the email)
2. Geolocate the flats (googlemaps library)
3. Extract informations from house description
4. Optimize informations for visualization
5. Visualize as an interactive map with HTML+JS (and save)

*NOTA BENE* in the GitHub repo I use an `example.dat` flats description file stripped of any real flat information.
It contains only two fictious flats: one in Tour Eiffel, and another in the Pantheon.

## 0. Setup modules 

In [1]:
import googlemaps
# from datetime import datetime
import json

# Google Maps User API key
mykey=open('gmaps.key').read().strip() # not shared on GitHub
kword_house='**** HOUSING'
kword_description='DESCRIPTION OF THE HOUSING'
fnout = 'example_map.html'

gmaps = googlemaps.Client(key=mykey)

## 1. Parse the informations

In [2]:
# Load email content from file (presaved with copy&paste)
with open('data/list_example.dat','r') as indata:
    data=[j for j in indata.readlines() if j.strip()]

In [3]:
houses=[]
addresses=[]
house=[]
address=''
isaddress=True

for j in data:
    if kword_house in j:
        # append old house to houses
        if house:
            houses.append(house)
            isaddress=True
        # then make a new house
        house=[j,]
    elif isaddress:
        if kword_description in j:
            isaddress=False
            addresses.append(address.strip())
            address=''
        else:
            address+=' '+j.strip()
        # append house description
        house.append(j)
    else:
        house.append(j)
houses.append(house)

In [4]:
## Test
# for j in xrange(max(len(houses),len(address))):
#        print ' '.join([ q.strip() for q in houses[j][1:3]])
#        print addresses[j]

## 2. Geolocate the flats

In [5]:
geoloces=[]
for j in addresses:
    geoloc=gmaps.geocode(j)
    geoloces.append(geoloc)

In [6]:
lats=[]
lngs=[]
ll=[]

import geojson
for j in geoloces:
    lat=j[0]['geometry']['location']['lat']
    lng=j[0]['geometry']['location']['lng']
    lats.append(lat)
    lngs.append(lng)

 
# average flats postion to center the map
mlat=sum(lats)/len(geoloces)
mlng=sum(lngs)/len(geoloces)

## 3. & 4. Extract and optimize informations

In [7]:
import re
housedescriptions=[]

euro="&#8364;"
sqm=" m<sup>2</sup>"
boldify = lambda x: "<b>"+x+"</b>"
endl = "<br/>"
hsep=' -- '

def failsafeRE(restr,instr,grp=0,default=''):
    try:
        return re.search(restr,instr).group(grp)
    except Exception,msg:
#         print repr(msg),instr,restr
        return default
    
for j,la,ln in zip(houses,lats,lngs):
    tmp=r''.join(j)
    
    try:

        link_url = link = failsafeRE('http.+',tmp,0)
        link = '<a href="{0:s}" target="_blank">photos</a>'.format(link)
        link2 = "https://www.google.com/maps/preview/@{latitude:f},{longitude:f},13z".format(latitude=la,longitude=ln)
        link2 = '<a href="{0:s}" target="_blank">gmap</a>'.format(link2)
        iframe = '<iframe src="{0:s}" width="100%" height="400px" frameborder="0"></iframe>'.format(link_url)
        
        rent = failsafeRE('(Monthly rent:)(.+)(euros)',tmp,2)+euro
        surface = failsafeRE('(Living space:)(.+)( sq.m)',tmp,2)+sqm
        station = failsafeRE('(Nearest Station:)(.+)',tmp,2)
        
        rs=boldify(rent+hsep+surface +hsep+station) +hsep + link+' '+link2+endl+iframe+endl
        tmp=rs+tmp
        
        tobolden=re.findall(r'(?m)^[A-Z][A-Z0-9 \':]+',tmp)
        for tb in tobolden:
            tmp=tmp.replace(tb,boldify(tb))
        
    except AttributeError,msg:
        pass
    
    housedescriptions.append(tmp)
#     print j[-6:]
len(housedescriptions)

2

## 5. Visualize as interactive map
### 5.1 Prepare geoJSON data

In [8]:
for lat,lng,h in zip(lats,lngs,housedescriptions):
    ll.append(geojson.Feature(geometry=geojson.Point([lng,lat]),properties={'popupContent':h.replace('\n','<br/>')}))
    
GJSN=geojson.FeatureCollection(ll)
data=geojson.dumps(GJSN)

In [9]:
# with open('houses.geojson', 'w') as fout:
#     fout.write(data)

### 5.2 Fill in the HTML template
I use `leaflet` to visualize the map and configure the map interaction

In [10]:
html=u"""
<link rel="stylesheet" href="http://cdn.leafletjs.com/leaflet/v0.7.7/leaflet.css" />
<link rel="stylesheet" type="text/css"
          href="https://fonts.googleapis.com/css?family=Open+Sans">
<style>
  body {{
    font-family: 'Open Sans', sans;
    font-size: 12px;
  }}
  #map {{
    width:66%;
    height:100%;
    padding:0px !important;
    margin:0px !important;
    top:0px !important;
    left:0px !important;
    position: fixed !important;
    display:block !important;
  }}

 div.leaflet-popup-content{{
     width:90ex !important;
     height:500px;
     overflow-y:scroll;
 }}
 #feature_infos {{
    width:33%;
    height: 100%;
    background: rgba(255, 255, 255, 1);
    position: absolute !important;
    top:0px;
    right:0px;
    vertical-align: middle;
    margin:0px !important;
    border:0px solid grey;
 }}
</style>
 
<div id="map"></div>
<div id="feature_infos"><span id="info"></span></div>

<script src="http://cdn.leafletjs.com/leaflet/v0.7.7/leaflet.js"></script>
<script>
    var map = L.map('map').setView([{cx:f}, {cy:f}], 13);

    function onEachFeature(feature, layer) {{    
        layer.on('mouseover', function (e) {{
            document.getElementById("info").innerHTML = feature.properties.popupContent;     
        }});
                
    }}
        
    L.tileLayer('http://{{s}}.tile.osm.org/{{z}}/{{x}}/{{y}}.png', 
        {{  maxZoom: 18,
            attribution: 'Map data &copy; <a href="http://openstreetmap.org">OpenStreetMap</a> contributors',
        }}).addTo(map);
        var data={data:s}
        var geojsonMarkerOptions = {{
            radius: 8,
            fillColor: "#ff7800",
            color: "#000",
            weight: 1,
            opacity: 1,
            fillOpacity: 0.8
        }};
        L.geoJson(data,{{onEachFeature : onEachFeature}}).addTo(map);
	</script>
""".format(data=data,cx=mlat,cy=mlng)


### 5.3 Save and show

In [11]:
from IPython.display import HTML

with open(fnout,'w') as fout:
    fout.write('<html><head/><body>{0:s}</body><html/>'.format(html).replace('500px','100%'))
HTML('<a href="{0:s}" target="_blank">{0:s}</a>'.format(fnout))

In [12]:
HTML('<iframe href="{0:s}" src="{0:s}" width="100%" height="500px"/>'.format(fnout))